In [6]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [7]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 1.2MB/s 
     |████████████████████████████████| 368kB 24.7MB/s 
     |████████████████████████████████| 3.2MB 43.9MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2


In [8]:
!pip install nltk

In [9]:
!pip install tflearn

     |████████████████████████████████| 102kB 2.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=cd58bd20e1300da3672e1bfac15c15bd177946c00420b5ab7f15f140cb2bbb61
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [10]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n\r\n        {"tag": "salutation",\r\n         "patterns": ["Salut", "Comment vas-tu", "\xc3\xa7a va ?", "Bonjour", "hello"],\r\n         "responses": ["Hello", "Je vais bien, merci", "\xc3\xa7a va oklm", "Bonjour", "Hi"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "au revoir",\r\n         "patterns": ["Bye", "Au revoir", "\xc3\xa0 plus tard", "ciao"],\r\n         "responses": ["Au plaisir, merci", "Bonne journ\xc3\xa9e", "Bye! Reviens vite."]\r\n        },\r\n        {"tag": "merci",\r\n         "patterns": ["Merci", "Je te remercie", "Merci beaucoup"],\r\n         "responses": ["Ravi de t\'avoir aid\xc3\xa9", "De rien!", "Avec plaisir"]\r\n        },\r\n        {"tag": "coronavirus",\r\n         "patterns": ["Qu\xe2\x80\x99est-ce qu\xe2\x80\x99un coronavirus ?", "C\'est quoi le coronavirus ?", "C\'est quoi un coronavirus ?", "C\'est quoi le corona ?", "Qu\'est ce qu\'un corona ?"],\r\n         "responses": ["Les coronavirus forme

In [0]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data, strict=False)

In [14]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Salut', 'Comment vas-tu', 'ça va ?', 'Bonjour', 'hello'],
   'responses': ['Hello',
    'Je vais bien, merci',
    'ça va oklm',
    'Bonjour',
    'Hi'],
   'tag': 'salutation'},
  {'patterns': ['Bye', 'Au revoir', 'à plus tard', 'ciao'],
   'responses': ['Au plaisir, merci', 'Bonne journée', 'Bye! Reviens vite.'],
   'tag': 'au revoir'},
  {'patterns': ['Merci', 'Je te remercie', 'Merci beaucoup'],
   'responses': ["Ravi de t'avoir aidé", 'De rien!', 'Avec plaisir'],
   'tag': 'merci'},
  {'patterns': ['Qu’est-ce qu’un coronavirus ?',
    "C'est quoi le coronavirus ?",
    "C'est quoi un coronavirus ?",
    "C'est quoi le corona ?",
    "Qu'est ce qu'un corona ?"],
   'responses': ['Les coronavirus forment une vaste famille de virus qui peuvent être pathogènes chez l’animal ou chez l’homme. On sait que, chez l’être humain, plusieurs coronavirus peuvent entraîner des infections respiratoires dont les manifestations vont du simple rhume

In [0]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

114 documents
28 classes ['COVID-19', 'animaux', 'antibio', 'au revoir', 'consultation', 'contact', 'contraction', 'coronavirus', 'courses', 'différence', 'domestiques', 'enfants', 'exposition', 'fécales', 'isolement', 'manger', 'masque', 'merci', 'porter', 'propagation', 'propage', 'protéger', 'quarantaine', 'respiration', 'salutation', 'symptômes', 'temps', 'traitement']
162 unique stemmed words [',', '19', 'a', 'ado', 'adolesc', 'anim', 'animaux', 'antibiot', 'au', 'aucun', 'aut', 'auto-quarantaine', 'avec', 'avoir', 'beaucoup', 'bonjo', 'bye', "c'est", 'cas', 'ce', 'ciao', 'com', 'comby', 'compagny', 'consult', 'cont', 'contact', 'contaminé', 'contract', 'coron', 'coronavir', 'correct', 'cours', 'covid', 'covid-19', "d'infection", "d'une", 'da', 'dan', 'de', 'des', 'difficulté', 'différ', 'dist', 'dois-je', 'domest', 'doul', 'du', 'efficac', 'en', 'enf', 'ent', 'est', 'est-ce', 'est-il', 'et', 'existe-t-il', 'exposé', 'fair', 'fruit', 'fécales', 'hello', 'infecté', 'isol', "j'ai", 

In [0]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [18]:
# resetting underlying graph data
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 28, activation="relu")
net = tflearn.fully_connected(net, 28, activation="relu")
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=2500, batch_size=28, show_metric=True)
model.save('model.tflearn')

Training Step: 12499  | total loss: 0.38917 | time: 0.017s
| Adam | epoch: 2500 | loss: 0.38917 - acc: 0.9316 -- iter: 112/114
Training Step: 12500  | total loss: 0.36207 | time: 0.022s
| Adam | epoch: 2500 | loss: 0.36207 - acc: 0.9349 -- iter: 114/114
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [0]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [0]:
with open('intents.json') as json_data:
    intents = json.load(json_data, strict=False)

In [23]:
# load the saved model
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [26]:
print("inpu_tag, taux de confiance :",classify('Bonjour?'))

inpu_tag, taux de confiance : [('salutation', 0.9133561)]


In [0]:
print("response, taux de confiance :",response('Bonjour?'))

In [27]:
response('Qu’est-ce que la COVID-19 ?')

La COVID-19 est la maladie infectieuse causée par le dernier coronavirus qui a été découvert. Ce nouveau virus et cette maladie étaient inconnus avant l’apparition de la flambée à Wuhan (Chine) en décembre 2019. La COVID-19 est maintenant pandémique et touche de nombreux pays dans le monde.


In [0]:
#Some of other context free responses.
response('Quels sont les symptômes de la COVID-19 ?')

In [0]:
response('Merci')

In [0]:
response('Bye')

In [31]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        else:
          print("tag, taux de confiance :", classify(inp))
          print(response(inp))


    
       
chat()

Start talking with the bot (type quit to stop)!
You: salut
tag, taux de confiance : [('salutation', 0.9272514)]
Bonjour
None
You: hello
tag, taux de confiance : [('salutation', 0.88848704)]
Je vais bien, merci
None
You: quit
